rewrite a ann_text which return dep_list I can direct put into graph

In [1]:
import spacy
import neuralcoref
from spacy import displacy # https://spacy.io/usage/visualizers
def show_depparse(text):
    doc = nlp(text)
    displacy.render(doc, style='dep',jupyter=True)

nlp = spacy.load("en")

##### remove_pronoun

##### dep_create

In [2]:
def ann_text(text):
    doc = nlp(text)
    for sent_idx, sent in enumerate(doc.sents):
        for tok_idx, tok in enumerate(sent):
            print(tok.lemma_, tok.pos_)
        print("===== next semt =====")

t = "The body of a fighter pilot who had gone missing following a mid-air collision last month was yesterday found near a fishing port in Pingtung County, the air force said."
ann_text(t)

the DET
body NOUN
of ADP
a DET
fighter NOUN
pilot NOUN
who PRON
have VERB
go VERB
miss VERB
follow VERB
a DET
mid ADJ
- NOUN
air NOUN
collision NOUN
last ADJ
month NOUN
be VERB
yesterday NOUN
find VERB
near ADP
a DET
fishing NOUN
port NOUN
in ADP
Pingtung PROPN
County PROPN
, PUNCT
the DET
air NOUN
force NOUN
say VERB
. PUNCT
===== next semt =====


In [41]:
import spacy
nlp = spacy.load('en')

# Add neural coref to SpaCy's pipe
import neuralcoref
neuralcoref.add_to_pipe(nlp)

#t = "Pan’s body was identified through its air force uniform, name tag and other items, the office said, adding that DNA testing would confirm the identity."
#t = "Angela lives in Boston. She is quite happy in that city."
doc2 = nlp(t)
for ent in doc2.ents:
    print(ent, ent._.coref_cluster)

print('=====')
doc = nlp(t)
print(doc._.has_coref)
print(doc._.coref_clusters)

Pan Ying-chun Pan Ying-chun: [Pan Ying-chun, him]
Pan’s None
yesterday None
=====
True
[the body: [the body, the body], Pan Ying-chun: [Pan Ying-chun, him], Pan’s brother: [Pan’s brother, he, he, his, his brother, him]]


In [37]:
type(doc._.coref_clusters[0])

neuralcoref.neuralcoref.Cluster

In [40]:
t = "“Since many of the things found on the body belonged to Pan Ying-chun, it is probably him,” Pan’s brother told reporters at the morgue yesterday. Finding the body was enough, he said, adding that he just wants to give his brother a proper funeral and send him off."

In [15]:
neuralcoref.add_to_pipe(nlp)
# t = 'Angela lives in Boston. She is quite happy in that city. She enjoy her life.'
doc = nlp(t)

cvt_dict = dict()

for s in doc._.coref_clusters:
    cvt_dict[str(s.main)] = [str(i) for i in s.mentions if i != s.main]
print(cvt_dict)

pn_list = []
for pn_main in cvt_dict:
    for pn in cvt_dict[pn_main]:
        pn_list.append( (pn, pn_main) )
print(pn_list)

no_pronoun_t = []
for tok in doc:
    if tok.lemma_ == "-PRON-":
        for pn_idx, pn in enumerate(pn_list):
            if tok.text == pn[0]:
                no_pronoun_t.append(pn[1])
                del(pn_list[pn_idx])
    else:
        no_pronoun_t.append(tok.text)
new_t = " ".join(no_pronoun_t)
new_t
nlp.remove_pipe("neuralcoref")
# for main_pn in cvt_dict:
#     for pn in cvt_dict[main_pn]:
#         t = t.replace(pn, main_pn)


# for tok in doc:
#     if tok.lemma_ == "-PRON-":
#         print(tok.text)
#     else:
#         print(tok.lemma_)

# print(doc._.coref_clusters)
# print(doc._.coref_clusters[1].mentions)
# print(doc._.coref_clusters[1].mentions[-1])
# print(doc._.coref_clusters[1].mentions[-1]._.coref_cluster.main)

# token = doc[-1]
# print("!", token)
# print(token._.in_coref)
# print(token._.coref_clusters[0])

# span = doc[-1:]
# print("!", span)
# print(span._.is_coref)
# print(span._.coref_cluster.main)
# print(span._.coref_cluster.main._.coref_cluster)

{'Lo Shang-hua (羅尚樺)': ['his', 'he', 'Lo'], 'his aircraft': ['his aircraft'], 'Pan’s': ['his', 'He', 'his'], 'The Taitung District Prosecutors’ Office': ['it', 'its']}
[('his', 'Lo Shang-hua (羅尚樺)'), ('he', 'Lo Shang-hua (羅尚樺)'), ('Lo', 'Lo Shang-hua (羅尚樺)'), ('his aircraft', 'his aircraft'), ('his', 'Pan’s'), ('He', 'Pan’s'), ('his', 'Pan’s'), ('it', 'The Taitung District Prosecutors’ Office'), ('its', 'The Taitung District Prosecutors’ Office')]


('neuralcoref', <neuralcoref.neuralcoref.NeuralCoref at 0x7fafd846c310>)

In [16]:
print(t)
print(new_t)

The other pilot, Lo Shang-hua (羅尚樺), ejected from his aircraft after the collision, but he did not have any vital signs when found at sea and was pronounced dead after being taken to hospital later that day. Lo was posthumously promoted from the rank of first lieutenant to major. Pan’s body has been transported to his home in Taitung County, the air force said. He is thought to have also ejected from his aircraft. The Taitung District Prosecutors’ Office said that it is continuing its investigation into the cause of the accident.
The other pilot , Lo Shang - hua ( 羅尚樺 ) , ejected from Lo Shang-hua (羅尚樺) Pan’s Pan’s aircraft after the collision , but Lo Shang-hua (羅尚樺) did not have any vital signs when found at sea and was pronounced dead after being taken to hospital later that day . Lo was posthumously promoted from the rank of first lieutenant to major . Pan ’s body has been transported to home in Taitung County , the air force said . Pan’s is thought to have also ejected from aircra

In [41]:
def remove_pronoun(input_t): # replace pronoun with original noun
    neuralcoref.add_to_pipe(nlp)
    doc = nlp(input_t)
    cvt_dict = dict()
    for s in doc._.coref_clusters: # find pronouns save it as dict
        cvt_dict[str(s.main)] = [str(i) for i in s.mentions if i != s.main]
#    print("pronoun dict:", cvt_dict)

    pn_list = [] # save list as (pronoun, origin noun)
    for pn_main in cvt_dict:
        for pn in cvt_dict[pn_main]:
            pn_list.append( (pn, pn_main) )
    rm_dup_pronouns(pn_list)
#    print("pronoun list without dup:", pn_list)
    
    no_pronoun_t = [] # replace pronoun with origin noun
    for tok in doc:
        if tok.lemma_ == "-PRON-":
            for pn_idx, pn in enumerate(pn_list):
                if tok.text == pn[0]:
                    no_pronoun_t.append(pn[1])
                    del(pn_list[pn_idx])
        else:
            no_pronoun_t.append(tok.text)
    new_t = " ".join(no_pronoun_t)
    nlp.remove_pipe("neuralcoref")
    return new_t

def rm_dup_pronouns(pn_list):
    rm_idx_list = []
    for idx, i in enumerate(pn_list):
        if i[0] == i[1]:
            rm_idx_list.append(idx)
    rm_idx_list.reverse()
    for rm_idx in rm_idx_list:
        del pn_list[rm_idx]
    # # example
    # pn_list = [('his', 'Lo Shang-hua (羅尚樺)'),("it","it"), ("the","the"), ('he', 'Lo Shang-hua (羅尚樺)'), ('Lo', 'Lo Shang-hua (羅尚樺)'), ('his aircraft', 'his aircraft'), ('his', 'Pan’s'), ('He', 'Pan’s'), ('his', 'Pan’s'), ('it', 'The Taitung District Prosecutors’ Office'), ('its', 'The Taitung District Prosecutors’ Office')]
    # rm_dup_pronouns(pn_list)
    # pn_list
    
t = 'Angela lives in Boston. She is quite happy in that city. She enjoy her life.'
remove_pronoun(t)

'Angela lives in Boston . Angela is quite happy in that city . Angela enjoy Angela life .'